In [2]:
import pandas as pd
import numpy as np
from  sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder # for one hot encoding , means for categorical data , it does not matter if the data is ordinal or nominal
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df = pd.read_csv('titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [5]:
x_train , x_test , y_train , y_test = train_test_split(df[['Pclass','Sex','Age']],df['Survived'],test_size=0.2,random_state=0) # [:,:-1 ] means all rows and all columns except the last column
x_train.head()

,Pclass,Sex,Age
20,1,male,55.0
306,1,male,30.0
142,1,male,61.0
14,1,female,47.0
284,3,female,2.0


In [26]:
trf1 = ColumnTransformer(transformers=[
    ('imputer',SimpleImputer(strategy='mean'),['Age']),
    ('ordinal',OrdinalEncoder(),['Sex']),
],
    remainder='passthrough')

In [27]:
trf1

ColumnTransformer(remainder='passthrough',
                  transformers=[('imputer', SimpleImputer(), ['Age']),
                                ('ordinal', OrdinalEncoder(), ['Sex'])])

In [40]:
x_train_transformed = trf1.fit_transform(x_train)
x_test_transformed = trf1.transform(x_test)


In [41]:
x_test_transformed_seen = pd.DataFrame(x_test_transformed, columns=x_test.columns)
x_test_transformed_seen.head()

,Pclass,Sex,Age
0,14.50000,1.0,3.0
1,30.42037,1.0,3.0
2,53.00000,0.0,1.0
3,30.42037,1.0,3.0
4,45.00000,0.0,3.0


In [42]:
# now we will use the pipeline to combine the column transformer and the model
pipe = Pipeline([
    ('preprocessing',trf1),
    ('model',LinearRegression())
])


/home/mavrick/anaconda3/lib/python3.12/site-packages/sklearn/compose/_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('imputer', SimpleImputer(),
                                                  ['Age']),
                                                 ('ordinal', OrdinalEncoder(),
                                                  ['Sex'])])),
                ('model', LinearRegression())])

ValueError: Specifying the columns using strings is only supported for dataframes.

In [38]:
# accuracy
pipe.score(x_train,y_train)
# 1.0 means 100% accuracy

1.0